In [2]:
!pip install -q -U tensorflow-addons

     |████████████████████████████████| 1.1MB 6.2MB/s 


In [6]:
__author__ = 'Alexander Soroka, soroka.a.m@gmail.com'
__copyright__ = """Copyright 2020 Alexander Soroka"""


import argparse
import glob
import numpy as np
import tensorflow as tf
import time
from tensorflow.python import keras as keras
from tensorflow.python.keras.callbacks import LearningRateScheduler

import tensorflow_addons as tfa


LOG_DIR = 'logs'
SHUFFLE_BUFFER = 4
BATCH_SIZE = 32
NUM_CLASSES = 6
PARALLEL_CALLS=4
RESIZE_TO = 224
TRAINSET_SIZE = 14034
VALSET_SIZE = 3000


def parse_proto_example(proto):
    keys_to_features = {
        'image/encoded': tf.io.FixedLenFeature((), tf.string, default_value=''),
        'image/class/label': tf.io.FixedLenFeature([], tf.int64, default_value=tf.zeros([], dtype=tf.int64))
    }
    example = tf.io.parse_single_example(proto, keys_to_features)
    example['image'] = tf.image.decode_jpeg(example['image/encoded'], channels=3)
    example['image'] = tf.image.convert_image_dtype(example['image'], dtype=tf.float32)
    example['image'] = tf.image.resize(example['image'], tf.constant([RESIZE_TO, RESIZE_TO]))
    return example['image'], tf.one_hot(example['image/class/label'], depth=NUM_CLASSES)


def normalize(image, label):
    return tf.image.per_image_standardization(image), label

def resize(image, label):
    return tf.image.resize(image, tf.constant([RESIZE_TO, RESIZE_TO])), label

def augmentation(image, label):
    degree = 15
    degrand = np.random.uniform(-degree, degree)
    image = tfa.image.rotate(image, np.pi * degrand / 180, interpolation='BILINEAR')
    return image, label

def create_dataset(filenames, batch_size):
    """Create dataset from tfrecords file
    :tfrecords_files: Mask to collect tfrecords file of dataset
    :returns: tf.data.Dataset
    """
    return tf.data.TFRecordDataset(filenames)\
        .map(parse_proto_example)\
        .map(resize)\
        .map(normalize)\
        .batch(batch_size)\
        .prefetch(batch_size)

def create_aug_dataset(filenames, batch_size):
    """Create dataset from tfrecords file
    :tfrecords_files: Mask to collect tfrecords file of dataset
    :returns: tf.data.Dataset
    """
    return tf.data.TFRecordDataset(filenames)\
        .map(parse_proto_example)\
        .map(augmentation)\
        .map(resize)\
        .map(normalize)\
        .shuffle(buffer_size=5 * batch_size)\
        .batch(batch_size)\
        .prefetch(2 * batch_size)


def build_model():
    new_model = tf.keras.applications.MobileNetV2(input_shape=(224,224,3),
						classes = NUM_CLASSES,
            include_top=False,
            weights='imagenet')
    
    return tf.keras.models.Sequential([
    	new_model,
      tf.keras.layers.GlobalAveragePooling2D(),
      tf.keras.layers.Dropout(0.3),
    	tf.keras.layers.Flatten(),
      tf.keras.layers.Dense(NUM_CLASSES, activation=tf.keras.activations.softmax)
    ])


def main():
    #args = argparse.ArgumentParser()
    #args.add_argument('--train', type=str, help='Glob pattern to collect train tfrecord files')
    #args.add_argument('--test', type=str, help='Glob pattern to collect test tfrecord files')
    #args = args.parse_args()
    train_path = '/content/drive/My Drive/SMOMI/dataset/train*'
    test_path = '/content/drive/My Drive/SMOMI/dataset/val*'

    train_dataset = create_aug_dataset(glob.glob(train_path), BATCH_SIZE)
    validation_dataset = create_dataset(glob.glob(test_path), BATCH_SIZE)

    model = build_model()
    model.load_weights('/content/drive/My Drive/SMOMI/w-4_2-15.hdf5')

    model.compile(
        optimizer=tf.optimizers.Adam(lr=1e-6),
        loss=tf.keras.losses.categorical_crossentropy,
        metrics=[tf.keras.metrics.categorical_accuracy],
    )

    log_dir='/content/drive/My Drive/SMOMI/{}/lab4_2-15/ilcd-{}'.format(LOG_DIR, time.time())
    weights_file = "/content/drive/My Drive/SMOMI/w-4_2-15.hdf5"

    model.fit(
        train_dataset,
        epochs=100,
        validation_data=validation_dataset,
        callbacks=[
            tf.keras.callbacks.TensorBoard(log_dir),
            tf.keras.callbacks.ModelCheckpoint(filepath=weights_file, monitor='val_categorical_accuracy', mode='max', save_best_only=True, save_weights_only=True, verbose=1)
        ]
    )


if __name__ == '__main__':
    main()

Epoch 1/100
      2/Unknown - 2s 821ms/step - loss: 0.6730 - categorical_accuracy: 0.8438WARNING:tensorflow:Callbacks method `on_train_batch_end` is slow compared to the batch time (batch time: 0.1329s vs `on_train_batch_end` time: 1.4557s). Check your callbacks.
    439/Unknown - 103s 234ms/step - loss: 0.4663 - categorical_accuracy: 0.7236
Epoch 00001: val_categorical_accuracy improved from -inf to 0.56033, saving model to /content/drive/My Drive/SMOMI/w-4_2-15.hdf5
439/439 [==============================] - 111s 252ms/step - loss: 0.4663 - categorical_accuracy: 0.7236 - val_loss: 0.9143 - val_categorical_accuracy: 0.5603
Epoch 2/100
439/439 [==============================] - ETA: 0s - loss: 0.4312 - categorical_accuracy: 0.7296
Epoch 00002: val_categorical_accuracy did not improve from 0.56033
439/439 [==============================] - 108s 246ms/step - loss: 0.4312 - categorical_accuracy: 0.7296 - val_loss: 0.9112 - val_categorical_accuracy: 0.5570
Epoch 3/100
439/439 [============

In [ ]:
__author__ = 'Alexander Soroka, soroka.a.m@gmail.com'
__copyright__ = """Copyright 2020 Alexander Soroka"""


import argparse
import glob
import numpy as np
import tensorflow as tf
import time
from tensorflow.python import keras as keras
from tensorflow.python.keras.callbacks import LearningRateScheduler

import tensorflow_addons as tfa


LOG_DIR = 'logs'
SHUFFLE_BUFFER = 4
BATCH_SIZE = 32
NUM_CLASSES = 6
PARALLEL_CALLS=4
RESIZE_TO = 224
TRAINSET_SIZE = 14034
VALSET_SIZE = 3000


def parse_proto_example(proto):
    keys_to_features = {
        'image/encoded': tf.io.FixedLenFeature((), tf.string, default_value=''),
        'image/class/label': tf.io.FixedLenFeature([], tf.int64, default_value=tf.zeros([], dtype=tf.int64))
    }
    example = tf.io.parse_single_example(proto, keys_to_features)
    example['image'] = tf.image.decode_jpeg(example['image/encoded'], channels=3)
    example['image'] = tf.image.convert_image_dtype(example['image'], dtype=tf.float32)
    example['image'] = tf.image.resize(example['image'], tf.constant([RESIZE_TO, RESIZE_TO]))
    return example['image'], tf.one_hot(example['image/class/label'], depth=NUM_CLASSES)


def normalize(image, label):
    return tf.image.per_image_standardization(image), label

def resize(image, label):
    return tf.image.resize(image, tf.constant([RESIZE_TO, RESIZE_TO])), label

def augmentation(image, label):
    degree = 45
    degrand = np.random.uniform(-degree, degree)
    image = tfa.image.rotate(image, np.pi * degrand / 180, interpolation='BILINEAR')
    return image, label

def create_dataset(filenames, batch_size):
    """Create dataset from tfrecords file
    :tfrecords_files: Mask to collect tfrecords file of dataset
    :returns: tf.data.Dataset
    """
    return tf.data.TFRecordDataset(filenames)\
        .map(parse_proto_example)\
        .map(resize)\
        .map(normalize)\
        .batch(batch_size)\
        .prefetch(batch_size)

def create_aug_dataset(filenames, batch_size):
    """Create dataset from tfrecords file
    :tfrecords_files: Mask to collect tfrecords file of dataset
    :returns: tf.data.Dataset
    """
    return tf.data.TFRecordDataset(filenames)\
        .map(parse_proto_example)\
        .map(augmentation)\
        .map(resize)\
        .map(normalize)\
        .shuffle(buffer_size=5 * batch_size)\
        .batch(batch_size)\
        .prefetch(2 * batch_size)


def build_model():
    new_model = tf.keras.applications.MobileNetV2(input_shape=(224,224,3),
						classes = NUM_CLASSES,
            include_top=False,
            weights='imagenet')
    
    return tf.keras.models.Sequential([
    	new_model,
      tf.keras.layers.GlobalAveragePooling2D(),
      tf.keras.layers.Dropout(0.3),
    	tf.keras.layers.Flatten(),
      tf.keras.layers.Dense(NUM_CLASSES, activation=tf.keras.activations.softmax)
    ])


def main():
    #args = argparse.ArgumentParser()
    #args.add_argument('--train', type=str, help='Glob pattern to collect train tfrecord files')
    #args.add_argument('--test', type=str, help='Glob pattern to collect test tfrecord files')
    #args = args.parse_args()
    train_path = '/content/drive/My Drive/SMOMI/dataset/train*'
    test_path = '/content/drive/My Drive/SMOMI/dataset/val*'

    train_dataset = create_aug_dataset(glob.glob(train_path), BATCH_SIZE)
    validation_dataset = create_dataset(glob.glob(test_path), BATCH_SIZE)

    model = build_model()

    model.compile(
        optimizer=tf.optimizers.Adam(lr=2e-6),
        loss=tf.keras.losses.categorical_crossentropy,
        metrics=[tf.keras.metrics.categorical_accuracy],
    )

    log_dir='/content/drive/My Drive/SMOMI/{}/lab4_2-45/ilcd-{}'.format(LOG_DIR, time.time())
    weights_file = "/content/drive/My Drive/SMOMI/w-4_2-45.hdf5"

    model.fit(
        train_dataset,
        epochs=150,
        validation_data=validation_dataset,
        callbacks=[
            tf.keras.callbacks.TensorBoard(log_dir),
            tf.keras.callbacks.ModelCheckpoint(filepath=weights_file, monitor='val_categorical_accuracy', mode='max', save_best_only=True, save_weights_only=True, verbose=1)
        ]
    )


if __name__ == '__main__':
    main()

Epoch 1/150
      2/Unknown - 1s 668ms/step - loss: 2.0044 - categorical_accuracy: 0.1094WARNING:tensorflow:Callbacks method `on_train_batch_end` is slow compared to the batch time (batch time: 0.1342s vs `on_train_batch_end` time: 1.1378s). Check your callbacks.
    439/Unknown - 100s 228ms/step - loss: 1.8923 - categorical_accuracy: 0.1459
Epoch 00001: val_categorical_accuracy improved from -inf to 0.08700, saving model to /content/drive/My Drive/SMOMI/w-4_2-45.hdf5
439/439 [==============================] - 108s 245ms/step - loss: 1.8923 - categorical_accuracy: 0.1459 - val_loss: 2.0658 - val_categorical_accuracy: 0.0870
Epoch 2/150
439/439 [==============================] - ETA: 0s - loss: 1.6512 - categorical_accuracy: 0.1659
Epoch 00002: val_categorical_accuracy improved from 0.08700 to 0.13267, saving model to /content/drive/My Drive/SMOMI/w-4_2-45.hdf5
439/439 [==============================] - 108s 246ms/step - loss: 1.6512 - categorical_accuracy: 0.1659 - val_loss: 1.7679 - v